## Imports

In [46]:
import numpy as np
import pandas as pd
import sbibm
import torch
import altair_saver
import tensorflow_probability as tfp
import tensorflow as tf
from sbibm.utils.kde import get_kde


from sbibm.metrics import c2st
from sbibm.visualisation import fig_posterior
from sbibm.metrics import c2st

## Thin function

In [48]:
def thin(X, length_out):
    assert length_out < X.shape[0]
    n = X.shape[0]
    keep = np.round(np.linspace(1, n, num = length_out)) - 1
    keep = keep.astype(int)
    return X[keep, :]

### Tasks and algorithms

In [49]:
task_names = ["gaussian_linear", "gaussian_linear_uniform", "gaussian_mixture", "bernoulli_glm", "sir"]
algorithm_names = ["rula", "rwm", "snle"]

## Posterior plots

In [8]:
plot_n = 1000
for task_name in task_names:
    for algorithm_name in algorithm_names:
        posterior_samples = np.genfromtxt(f"./samples/{task_name}_{algorithm_name}.txt")
        posterior_samples = thin(posterior_samples, plot_n)
        posterior_samples = torch.tensor(posterior_samples)        
        
        n_params = posterior_samples.shape[1] 
        fig_size = 400-35*n_params
        scatter_size = 3.5+0.05*n_params

        fig = fig_posterior(
            task_name=task_name,
            observation=1,
            samples_tensor = posterior_samples,
            num_samples = plot_n,
            config = "streamlit",
            height = fig_size,
            width = fig_size,
            scatter_size = scatter_size,
            samples_name = algorithm_name
        )
        altair_saver.save(fig, f"./plots/{task_name}_{algorithm_name}.html")

    

/home/dw16200/miniconda3/envs/snle_env/lib/python3.8/site-packages/diffeqtorch/diffeqtorch.py:29: UserWarning: JULIA_SYSIMAGE_DIFFEQTORCH not set
  warn("JULIA_SYSIMAGE_DIFFEQTORCH not set")


## Performance metrics
We will compute the classification accuracy twice. Once on all the samples (3000), then once on a thinned sample of 300 points. The size of the reference is matched to the size of the data to make sure the neural network did not have issues with the biased class sizes.

In [12]:
metrics = {
    "task": [],
    "algorithm":  [],
    "min_ess": [],
    "mean_ess": [],
    "max_ess": [],
    "c2st_all": [],
    # "c2st_thinned": [] Not too sure this is a good idea yet
}

tf.config.experimental.enable_tensor_float_32_execution(False)

for task_name in task_names:
    for algorithm_name in algorithm_names:
        print(f"{task_name}: {algorithm_name}")
        posterior_samples = torch.tensor(np.genfromtxt(f"./samples/{task_name}_{algorithm_name}.txt"))
        task = sbibm.get_task(task_name)
        reference_samples = task.get_reference_posterior_samples(num_observation=1)
        
        ess = tfp.mcmc.effective_sample_size(posterior_samples)

        X = posterior_samples
        Y = reference_samples[1:X.shape[0], :]
        c2st_all = c2st(X, Y)[0].item()
            
        metrics["task"].append(task_name)
        metrics["algorithm"].append(algorithm_name)
        metrics["min_ess"].append(np.min(ess))
        metrics["mean_ess"].append(np.mean(ess))
        metrics["max_ess"].append(np.max(ess))
        metrics["c2st_all"].append(c2st_all)
        
        # X = thin(posterior_samples, round(np.mean(ess)))
        # Y = reference_samples[1:X.shape[0], :]
        # c2st_thinned = c2st(X, Y)[0].item()  
        # metrics["c2st_thinned"].append(c2st_thinned)


gaussian_linear: rula
gaussian_linear: rwm
gaussian_linear: snle
gaussian_linear_uniform: rula
gaussian_linear_uniform: rwm
gaussian_linear_uniform: snle
gaussian_mixture: rula
gaussian_mixture: rwm
gaussian_mixture: snle
bernoulli_glm: rula
bernoulli_glm: rwm
bernoulli_glm: snle
sir: rula
sir: rwm
sir: snle


In [13]:
df = pd.DataFrame(metrics)

,task,algorithm,min_ess,mean_ess,max_ess,c2st_all
0,gaussian_linear,rula,85.272479,119.780321,175.290260,0.733291
1,gaussian_linear,rwm,46.749130,84.969092,129.220448,0.910484
2,gaussian_linear,snle,2783.865990,2937.087296,3000.000000,0.514918
3,gaussian_linear_uniform,rula,93.496974,152.492718,243.006795,0.755625
4,gaussian_linear_uniform,rwm,32.334730,50.081628,69.616370,0.927652
5,gaussian_linear_uniform,snle,2951.877323,2990.814054,3000.000000,0.506081
6,gaussian_mixture,rula,196.244477,260.601150,324.957823,0.781626
7,gaussian_mixture,rwm,317.945313,368.656782,419.368251,0.796301
8,gaussian_mixture,snle,2934.901045,2967.450523,3000.000000,0.578265
9,bernoulli_glm,rula,150.903994,210.484827,273.521679,0.699619


## Thinned C2ST

In [72]:
# mean_esses = df.groupby("task")["mean_ess"].min().round().astype(int)

thinned_c2st = {
    "task": [],
    "algorithm":  [],
    "c2st_thinned": []
}

for task_name in task_names:
    
    # n = mean_esses[task_name]
    n = 300
    
    for algorithm_name in algorithm_names:
        print(f"{task_name}: {algorithm_name}")
        posterior_samples = torch.tensor(np.genfromtxt(f"./samples/{task_name}_{algorithm_name}.txt"))
        task = sbibm.get_task(task_name)
        reference_samples = task.get_reference_posterior_samples(num_observation=1)
    
        X = thin(posterior_samples, n)
        Y = thin(reference_samples, n)

        result = c2st(X, Y)[0].item()
            
        thinned_c2st["task"].append(task_name)
        thinned_c2st["algorithm"].append(algorithm_name)
        thinned_c2st["c2st_thinned"].append(result)
    
        # X = thin(posterior_samples, round(np.mean(ess)))
        # c2st_thinned = c2st(X, Y)[0].item()  
        # metrics["c2st_thinned"].append(c2st_thinned)


gaussian_linear: rula
gaussian_linear: rwm
gaussian_linear: snle
gaussian_linear_uniform: rula
gaussian_linear_uniform: rwm
gaussian_linear_uniform: snle
gaussian_mixture: rula
gaussian_mixture: rwm
gaussian_mixture: snle
bernoulli_glm: rula
bernoulli_glm: rwm
bernoulli_glm: snle
sir: rula
sir: rwm
sir: snle


In [84]:
df = df.merge(pd.DataFrame(thinned_c2st))

### KDE C2ST

In [134]:
# mean_esses = df.groupby("task")["mean_ess"].min().round().astype(int)

kde_c2st = {
    "task": [],
    "algorithm":  [],
    "c2st_kde": []
}

for task_name in task_names:
    
    for algorithm_name in algorithm_names:
        print(f"{task_name}: {algorithm_name}")
        posterior_samples = torch.tensor(np.genfromtxt(f"./samples/{task_name}_{algorithm_name}.txt"))
        task = sbibm.get_task(task_name)
        reference_samples = task.get_reference_posterior_samples(num_observation=1)
        
        kde = get_kde(posterior_samples)
        X = kde.sample(3000)
        
        Y = thin(reference_samples, 3000)

        result = c2st(X, Y)[0].item()
            
        kde_c2st["task"].append(task_name)
        kde_c2st["algorithm"].append(algorithm_name)
        kde_c2st["c2st_kde"].append(result)


gaussian_linear: rula
gaussian_linear: rwm
gaussian_linear: snle
gaussian_linear_uniform: rula
gaussian_linear_uniform: rwm
gaussian_linear_uniform: snle
gaussian_mixture: rula


/home/dw16200/miniconda3/envs/snle_env/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [ -289.78870071  -293.68555975  -306.54238012  -365.38838532
 -1184.72573057            nan            nan            nan
            nan            nan]
  warnings.warn(
/home/dw16200/miniconda3/envs/snle_env/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [            nan             nan             nan             nan
             nan -25896.69724704  -2914.866925    -1185.40700707
   -723.96939396   -535.97608142]
  warnings.warn(


gaussian_mixture: rwm


/home/dw16200/miniconda3/envs/snle_env/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [-138.21037946 -138.52775017 -143.36492265 -168.06494724 -554.34673044
           nan           nan           nan           nan           nan]
  warnings.warn(
/home/dw16200/miniconda3/envs/snle_env/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [            nan             nan             nan             nan
             nan -13345.15914262  -1395.59014112   -554.34668057
   -331.6649022    -246.40449873]
  warnings.warn(


gaussian_mixture: snle


...

/home/dw16200/miniconda3/envs/snle_env/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [876.36207106 887.41339212 894.3327935  894.1997021  840.19341457
          nan          nan          nan          nan          nan]
  warnings.warn(


sir: rwm


/home/dw16200/miniconda3/envs/snle_env/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [  913.09523949   853.10288023   696.62851249   103.94675514
 -7701.89562008            nan            nan            nan
            nan            nan]
  warnings.warn(
/home/dw16200/miniconda3/envs/snle_env/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [             nan              nan              nan              nan
              nan -227079.66419743  -23691.6166227    -7696.70381797
   -3355.05894285   -1596.07458319]
  warnings.warn(


sir: snle


/home/dw16200/miniconda3/envs/snle_env/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [946.42746416 963.28208121 975.55179693 981.46402287 973.35318988
          nan          nan          nan          nan          nan]
  warnings.warn(


In [139]:
df = df.merge(pd.DataFrame(kde_c2st))

### Combine this with other results

In [81]:
rula_df = pd.read_csv("results/rula.csv")
rula_df["algorithm"] = "rula"

rwm_df = pd.read_csv("results/rwm.csv")
rwm_df["algorithm"] = "rwm"

snle_df = pd.read_csv("results/snle.csv", header=None, names = ["task", "run_time"])
snle_df["algorithm"] = "snle"

times_and_acceptance_rate_df = pd.concat([rula_df, rwm_df, snle_df])
df = df.merge(times_and_acceptance_rate_df, on = ["task", "algorithm"])
df

,task,algorithm,min_ess,mean_ess,max_ess,c2st_all,run_time_x,acceptance_rate_x,run_time_y,acceptance_rate_y,c2st_thinned,run_time,acceptance_rate
0,gaussian_linear,rula,85.272479,119.780321,175.290260,0.733291,1837.731509,NaN,1837.731509,NaN,0.516667,1837.731509,NaN
1,gaussian_linear,rwm,46.749130,84.969092,129.220448,0.910484,1524.177910,0.242000,1524.177910,0.242000,0.555000,1524.177910,0.242000
2,gaussian_linear,snle,2783.865990,2937.087296,3000.000000,0.514918,6630.660445,NaN,6630.660445,NaN,0.538333,6630.660445,NaN
3,gaussian_linear_uniform,rula,93.496974,152.492718,243.006795,0.755625,1931.989645,NaN,1931.989645,NaN,0.535000,1931.989645,NaN
4,gaussian_linear_uniform,rwm,32.334730,50.081628,69.616370,0.927652,694.577821,0.189000,694.577821,0.189000,0.671667,694.577821,0.189000
5,gaussian_linear_uniform,snle,2951.877323,2990.814054,3000.000000,0.506081,5860.031893,NaN,5860.031893,NaN,0.526667,5860.031893,NaN
6,gaussian_mixture,rula,196.244477,260.601150,324.957823,0.781626,1024.953490,NaN,1024.953490,NaN,0.781667,1024.953490,NaN
7,gaussian_mixture,rwm,317.945313,368.656782,419.368251,0.796301,727.444054,0.331000,727.444054,0.331000,0.770000,727.444054,0.331000
8,gaussian_mixture,snle,2934.901045,2967.450523,3000.000000,0.578265,3111.621493,NaN,3111.621493,NaN,0.556667,3111.621493,NaN
9,bernoulli_glm,rula,150.903994,210.484827,273.521679,0.699619,1348.098138,NaN,1348.098138,NaN,0.658333,1348.098138,NaN


In [ ]:
df.to_csv("results/general_performance.csv", index = False)

## Check KDE reasonable
Sanity check that KDE does not reduce performance significantly on true posterior samples

In [128]:
# mean_esses = df.groupby("task")["mean_ess"].min().round().astype(int)
kde_check = {
    "task": [],
    "no_kde_c2st": [],
    "kde_c2st": []
}

for task_name in task_names:
    
    task = sbibm.get_task(task_name)
    reference_samples = task.get_reference_posterior_samples(num_observation=1)
    
    X_no_kde = reference_samples[0:3000, :]
    kde = get_kde(X_no_kde)
    X_kde = kde.sample(3000)
    
    Y = reference_samples[7000:10000, :]
    kde_check["task"].append(task_name)
    kde_check["no_kde_c2st"].append(c2st(X_no_kde, Y)[0].item())
    kde_check["kde_c2st"].append(c2st(X_kde, Y)[0].item())
            

/home/dw16200/miniconda3/envs/snle_env/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [-118.78932776 -106.22307385 -102.08979517 -132.03950519 -702.59941455
           nan           nan           nan           nan           nan]
  warnings.warn(
/home/dw16200/miniconda3/envs/snle_env/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [  -2874.55063472   -4460.23988836   -8351.47436143  -24504.95499338
 -337142.58918713              nan              nan              nan
              nan              nan]
  warnings.warn(
/home/dw16200/miniconda3/envs/snle_env/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [               nan                nan                nan
                nan                nan -11373510.25109412
  -1088139.28293656   -336228.18

In [133]:
pd.DataFrame(kde_check).to_csv("results/kde_check.csv", index=False)